In [ ]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
from scipy import interpolate
import plotly.graph_objs as go
from ai_model.net import *
import math
from io import BytesIO
import csv
import sys
from .dataGene import *

In [ ]:
def odata_proceed(data):
    x=[float(i[0]) for i in data]
    y=[float(i[1]) for i in data]
    s=0
    e=len(x)-1
    for i in range(len(x)):
        if x[i]>0:
            s=i
            break
    for i in range(len(x)):
        if x[e-i]<12:
            e=e-i
            break
    x=x[s:e+1]
    y=y[s:e+1]
    for i in range(len(x)):
        if y[i]<0:
            y[i]=0
    for i in range(1000):
        y[i]=0
    x_1=np.linspace(0,12,60000)
    f=interpolate.interp1d(x,y,kind='cubic',bounds_error=False,fill_value=0)
    y_1=f(x_1)
    y_1=y_1/np.max(y_1)
    for i in range(len(y_1)):
        if y_1[i]<0.001:
            y_1[i]=0
    return y_1

In [ ]:
all_meta=[]
with open('./all_meta.txt') as f:
    lines=f.readlines()
    for k in lines:
        all_meta.append(k.strip())

In [ ]:
y_data=[]
filename1='' #your csv file
with open(filename1) as f:
    for line in f.readlines():
        y_data.append(line.replace('\n','').split('\t'))
y_data=odata_proceed(y_data)
filename2='' #your txt file
peaks=[]
with open(filename2, 'r') as file:
    for line in file:
        peaks.append(float(line.strip()))


In [ ]:
om=dataGene(allMeta=all_meta)

In [ ]:
output=om.test_m('./onedTrans_0.9710',y_data,peaks)
output.to_csv('output.csv')

In [ ]:
#plot
x_data = np.linspace(0,12,60000)
in_peaks0=np.array(output.peak)
labels0=np.array(output.pred_0)
trace1 = go.Scatter(x=x_data[::-1], y=y_data[::-1], mode='lines', name='Line',line=dict(color='black'))
trace2 = go.Scatter(x=in_peaks0[::-1], y=[y_data[int(in_peaks0[i]*5000)] for i in range(len(in_peaks0))][::-1], mode='markers', name='Markers', text=labels0[::-1], 
                    marker=dict(size=5))
data = [trace1, trace2]
layout = go.Layout(xaxis=dict(
        autorange='reversed'  
    ),title='result')
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=False),
    plot_bgcolor='white',
)